<h1 style="text-align: center; font-size: 60px;"> Интеллектуальный анализ и обработка данных</h1>
<hr>
<h2 style="text-align: center;font-size: 40px;"> ЛАБОРАТОРНАЯ РАБОТА №1 <br>
 Разведочный и предиктивный анализ данных на основе регрессионных моделей</h2>
 <hr>

<h3 style="text-align: center; font-size: 30px;">Исполнители</h3>
<p><b>Корзун Николай Вадимович</b> <a href="https://github.com/nikoloskorzun">GitHub</a></p>


<hr>
<h3 style="text-align: center; font-size: 30px;">Постановка задачи</h3>

<p>Дан многомерный размеченный набор данных. Необходимо выполнить предиктивный анализ данных на основе регрессионных моделей в соответствии со следующей последовательностью этапов.</p> 

<ol>
    <li>Загрузить необходимые пакеты и библиотеки.  </li>
    <li>Загрузить данные из указанного источника.  
    <li>Выполнить разведочный анализ данных в соответствии с этапами описанными в файле Этапы проекта машинного обучения в примерах 1/2.pdf: 
    <ol>
        <li>Ознакомление с данными с помощью методов описательной статистики;  </li>
        <li>Выполнить визуализацию данных одномерную для понимания распределения данных и многомерную для выяснения зависимостей между признаками;  </li>
        <li>При необходимости выполнить очистку данных одним из методов.  </li>
        <li>Проанализировать корреляционную зависимость между признаками;  </li>
        <li>Поэкспериментировать с комбинациями атрибутов. При необходимости добавить новые атрибуты в набор данных.  </li>
        <li>Выполнить отбор существенных признаков. Сформировать набор данных из существенных признаков.  </li>
        <li>При необходимости преобразовать текстовые или категориальные признаки одним из методов.  </li>
        <li>Выполнить масштабирование данных для обоих наборов (исходного и сформированного) одним из методов по варианту. </li>
    </ol>
    </li>
    <li>Анализ выполняется для исходного набора данных, масштабированного (п.3.h) набора исходного данных, построенного набора данных (удаление неинформативных, добавление пользовательских ризнаков, приведение нечисловых признаков к численному виду) и масштабированного построенного набора данных. Во всех наборах данных выделить обучающую и тестовую выборки данных.  </li>
<li>Сравнить линейную (по варианту) и полиномиальную регрессионные модели на обучающих выборках для всех наборов данных, включая их масштабированные варианты. Для оценки качества моделей использовать метрики: корень из среднеквадратичной ошибки, R2.  </li>
<li>Для лучшей модели на лучшем наборе данных выполнить Grid поиск лучшего гиперпараметра регрессионной модели на обучающей выборке. Построить графики результатов поиска. Определить значение лучшего гиперпараметра. </li> 
<li>Для лучшей модели на лучшем наборе данных выполнить Grid поиск лучшего гиперпараметра регрессионной модели на тестовом наборе. Построить графики результатов поиска. Скорректировать значение лучшего гиперпараметра модели.  </li>
<li>Сделать выводы по проведенному анализу.</li>
    
</ol>
<p><b>Вариант №12</b></p>
<p>Набор данных различных форм зданий, различающихся по площади остекления, распределению площади остекления, ориентации и другим параметрам. <br>В зависимости от параметров здания других признаков аргументов. Построить регрессионную модель оценки целевого признака требований к тепловой нагрузке зданий:</p>

<ol>
    <li>X1 вещественный, относительная компактность</li>
    <li>X2 вещественный, площадь поверхности</li>
    <li>X3 вещественный, площадь сплошной стены</li>
    <li>X4 вещественный, зона крыши</li>
    <li>X5 вещественный, общая высота</li>
    <li>X6 целочисленный, ориентация</li>
    <li>X7 вещественный, площадь сплошного остекления</li>
    <li>X8 целочисленный, распределение площади остекления</li>    
    <li>Y1 целевой признак, вещественный, отопительная нагрузка</li>
</ol>

a. Пункт 3.h – Нормализация
b. Пункт 4 – гребневая регрессия (регуляризованная линейная регрессия

<h3 style="text-align: center; font-size: 30px;">Выполнение задачи</h3>
<h4 style="text-align: center; font-size: 24px;">Этап 2<br>Загрузка датасета</h4>

In [1]:
names = ['относительная компактность', 'площадь поверхности', 'площадь сплошной стены',
         'зона крыши', 'общая высота', 'ориентация', 'площадь сплошного остекления',
         'распределение площади остекления', 'отопительная нагрузка']


In [2]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap


from IPython.core.display import HTML
from IPython.display import display

import ipywidgets as widgets


import pandas as pd

import seaborn as sns





import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px


from sklearn.feature_selection import SelectKBest, f_regression

from sklearn.linear_model import Ridge, LinearRegression
from sklearn.preprocessing import StandardScaler, Normalizer, OrdinalEncoder, PolynomialFeatures

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error, r2_score


In [5]:
global df
df = pd.read_excel("V12.xlsx", names = names)
global corr

<h4 style="text-align: center; font-size: 24px;">Этап 3<br>Разведочный анализ данных</h4>
<h5 style="text-align: center; font-size: 20px;">Описательная статистика</h5>

In [6]:
HTML(f'<p>размерность набора данных: <b>{df.shape}</b></p>')

In [7]:
print("Типы атрибутов:", end="\n\n")
df.info()
#df.head()

Типы атрибутов:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   относительная компактность        768 non-null    float64
 1   площадь поверхности               768 non-null    float64
 2   площадь сплошной стены            768 non-null    float64
 3   зона крыши                        768 non-null    float64
 4   общая высота                      768 non-null    float64
 5   ориентация                        768 non-null    int64  
 6   площадь сплошного остекления      768 non-null    float64
 7   распределение площади остекления  768 non-null    int64  
 8   отопительная нагрузка             768 non-null    float64
dtypes: float64(7), int64(2)
memory usage: 54.1 KB


In [8]:
pd.set_option('display.precision', 3)
df.describe()

,относительная компактность,площадь поверхности,площадь сплошной стены,зона крыши,общая высота,ориентация,площадь сплошного остекления,распределение площади остекления,отопительная нагрузка
count,768.000,768.000,768.000,768.000,768.000,768.000,768.000,768.000,768.000
mean,0.764,671.708,318.500,176.604,5.250,3.500,0.234,2.812,22.307
std,0.106,88.086,43.626,45.166,1.751,1.119,0.133,1.551,10.090
min,0.620,514.500,245.000,110.250,3.500,2.000,0.000,0.000,6.010
25%,0.682,606.375,294.000,140.875,3.500,2.750,0.100,1.750,12.992
50%,0.750,673.750,318.500,183.750,5.250,3.500,0.250,3.000,18.950
75%,0.830,741.125,343.000,220.500,7.000,4.250,0.400,4.000,31.668
max,0.980,808.500,416.500,220.500,7.000,5.000,0.400,5.000,43.100


<h5 style="text-align: center; font-size: 20px;">Визуализация данных</h5>

In [ ]:

# Функции для расчета количества бинов по разным правилам
def sturges_bins(data):
    n = len(data)
    return int(np.ceil(np.log2(n)) + 1)

def scott_bins(data):
    h = 3.5 * np.std(data) / (len(data) ** (1/3))
    return int(np.ceil((np.max(data) - np.min(data)) / h))

def fd_bins(data):
    iqr = np.percentile(data, 75) - np.percentile(data, 25)
    h = 2 * iqr / (len(data) ** (1/3))
    return int(np.ceil((np.max(data) - np.min(data)) / h))

def sqrt_bins(data):
    return int(np.ceil(np.sqrt(len(data))))

# Словарь с доступными правилами
rules_dict = {
    '100_n': lambda _: 100,
    'sturges': sturges_bins,
    'scott': scott_bins,
    'fd': fd_bins,
    'sqrt': sqrt_bins,
    '50_n': lambda _: 50,
    '20_n': lambda _: 20,
    '10_n': lambda _: 10,

}
# Создаем выпадающие списки для каждого столбца
rule_widgets = {column: widgets.Dropdown(options=list(rules_dict.keys()), description=f'{column}:') for column in df.columns}

# Функция для обновления графиков
def update_plots(**rules):
    n_cols = 3  # Количество графиков в ряду
    n_rows = (len(df.columns) + n_cols - 1) // n_cols  # Количество рядов
    
    # Создаем subplot с помощью Plotly
    fig = make_subplots(rows=n_rows, cols=n_cols, subplot_titles=df.columns)
    
    for i, column in enumerate(df.columns):
        row = (i // n_cols) + 1
        col = (i % n_cols) + 1
        
        rule = rules[column]  # Получаем правило для текущего столбца
        
        # Выбираем количество бинов в зависимости от правила

        bins = rules_dict[rule](df[column])
        #print(bins)
        # Строим гистограмму
        fig.add_trace(
            go.Histogram(x=df[column], nbinsx=bins, name=column),
            row=row, col=col
        )
    
    # Настраиваем макет
    fig.update_layout(
        height=n_rows * 400,  # Высота графика
        width=1200,  # Ширина графика
        showlegend=False,  # Скрываем легенду
        title_text="Одномерные графики - Гистограммы"
    )
    
    # Показываем график
    fig.show()

# Интерактивное обновление графиков
widgets.interact(update_plots, **rule_widgets)

interactive(children=(Dropdown(description='относительная компактность:', options=('100_n', 'sturges', 'scott'…

<function __main__.update_plots(**rules)>

<h6 style="text-align: left; font-size: 20px;">Одномерные графики - график плотности</h6>

In [10]:
def density(kind):
    df.plot(kind=kind, subplots=True, layout=(3, 3), sharex=False, legend=True, fontsize=1, figsize=(15,15))
    plt.show()
widgets.interact(density, kind=["density", "kde"]) #useless

interactive(children=(Dropdown(description='kind', options=('density', 'kde'), value='density'), Output()), _d…

<function __main__.density(kind)>

<h6 style="text-align: left; font-size: 20px;">Многомерные графики: график корреляционной матрицы.</h6>


In [12]:

def heatmap(method, **kwargs):
    selected_items = [item for item, selected in kwargs.items() if selected]
    global corr

    if len(selected_items) <= 1:
        corr = df.corr(method=method)

        return
    
    corr = df[selected_items].corr(method=method)

    
    colors = [(1, 1, 1), (0, 0, 0), (1, 1, 1)]  # Белый -> Черный -> Белый
    # Создаем цветовую карту
    cmap = LinearSegmentedColormap.from_list('custom_cmap', colors, N=256)
    plt.figure(figsize=(15, 15))
    plt.title(f"Матрицы корреляции method = {method}")
    sns.heatmap(corr, vmax=1, square=True, annot=True, cmap=cmap)
    plt.show()
    corr = df.corr(method=method)

widgets.interact(heatmap, method = ["pearson", "kendall", "spearman"], **{item: widgets.Checkbox(description=item, value=False) for item in list(df.columns)})


interactive(children=(Dropdown(description='method', options=('pearson', 'kendall', 'spearman'), value='pearso…

<function __main__.heatmap(method, **kwargs)>

<h6 style="text-align: left; font-size: 20px;">Многомерные графики: матрица диаграммы рассеяния.</h6>
<a id='scatter-diag-plots'></a>

In [ ]:

# Функция, которая будет вызвана при изменении состояния чекбоксов
def scatterplots(**kwargs):
    selected_items = [item for item, selected in kwargs.items() if selected]
    if len(selected_items) < 1:
        return
    pd.plotting.scatter_matrix(df[selected_items], figsize=(15, 15), diagonal='kde')
    plt.show()


widgets.interact(scatterplots, **{item: widgets.Checkbox(description=item, value=False) for item in list(df.columns)})  


interactive(children=(Checkbox(value=False, description='относительная компактность'), Checkbox(value=False, d…

<function __main__.scatterplots(**kwargs)>

<h6 style="text-align: left; font-size: 20px;">Многомерные графики: матрица диаграммы рассеяния.</h6>


In [14]:
new_corr = corr["отопительная нагрузка"].drop("отопительная нагрузка").sort_values(ascending=False, key=lambda x: x.abs())
print(new_corr)

общая высота                        0.889
зона крыши                         -0.862
площадь поверхности                -0.658
относительная компактность          0.622
площадь сплошной стены              0.456
площадь сплошного остекления        0.270
распределение площади остекления    0.087
ориентация                         -0.003
Name: отопительная нагрузка, dtype: float64


In [ ]:
widgets.interact(scatterplots, **{item: widgets.Checkbox(description=item, value=False) for item in list(df.columns)})  

interactive(children=(Checkbox(value=False, description='относительная компактность'), Checkbox(value=False, d…

<function __main__.scatterplots(**kwargs)>

<h6 style="text-align: left; font-size: 20px;">Экспериментирование с комбинациями атрибутов</h6>


In [16]:
df_base = df.copy()
df["отношение стенной площади ко всей площади дома"] = df["площадь сплошной стены"] / df["площадь поверхности"]
df["отношение остекленния ко всей площади дома"] = df["площадь сплошного остекления"] / df["площадь поверхности"]
df["отношение площади крыши ко всей площади дома"] = df["зона крыши"] / df["площадь поверхности"]
new_corr = corr["отопительная нагрузка"].drop("отопительная нагрузка").sort_values(ascending=False, key=lambda x: x.abs())
print(new_corr)

общая высота                        0.889
зона крыши                         -0.862
площадь поверхности                -0.658
относительная компактность          0.622
площадь сплошной стены              0.456
площадь сплошного остекления        0.270
распределение площади остекления    0.087
ориентация                         -0.003
Name: отопительная нагрузка, dtype: float64


In [ ]:
#widgets.interact(scatterplots, **{item: widgets.Checkbox(description=item, value=False) for item in list(df.columns)})
widgets.interact(heatmap, method = ["pearson", "kendall", "spearman"], **{item: widgets.Checkbox(description=item, value=False) for item in list(df.columns)})


interactive(children=(Dropdown(description='method', options=('pearson', 'kendall', 'spearman'), value='pearso…

<function __main__.heatmap(method, **kwargs)>

<h4 style="text-align: center; font-size: 24px;">Этап 4<br>Подготовка данных</h4>
<h5 style="text-align: center; font-size: 20px;">Очистка данных</h5>

In [18]:
if np.sum(df.describe().head(1).to_numpy())/(df.shape[0] * df.shape[1]) == 1: # ==1 bad idea
    print("Не требуется")
else:
    pass

Не требуется


<h5 style="text-align: center; font-size: 20px;">Отбор существенных(информативных) признаков</h5>

In [19]:
df_X = df.drop(columns='отопительная нагрузка')
df_Y = df['отопительная нагрузка']

In [20]:

# Масштабирование данных (гребневая регрессия чувствительна к масштабу данных)
scaler = StandardScaler()
df_X_scaled = scaler.fit_transform(df_X)

# Создание и обучение модели гребневой регрессии
ridge = Ridge(alpha=1.0)  # alpha — параметр регуляризации
ridge.fit(df_X_scaled, df_Y)

# Получение коэффициентов модели
coefficients = ridge.coef_

# Создание DataFrame с оценками важности признаков
df_scores = pd.DataFrame(coefficients, index=df_X.columns, columns=['Оценка'])

# Добавление коэффициентов корреляции (если они уже есть в вашем коде)
df_scores['Коэффициент корреляции'] = corr["отопительная нагрузка"].drop("отопительная нагрузка")

# Переименовываем индекс в 'Признак'
df_scores = df_scores.reset_index().rename(columns={'index': 'Признак'})

# Сортировка по абсолютным значениям оценок (важности признаков)
df_scores['Оценка'] = df_scores['Оценка'].abs()  # Используем абсолютные значения коэффициентов
df_scores = df_scores.sort_values(by='Оценка', ascending=True)

display(df_scores)

,Признак,Оценка,Коэффициент корреляции
5,ориентация,0.026,-0.003
7,распределение площади остекления,0.315,0.087
2,площадь сплошной стены,0.654,0.456
8,отношение стенной площади ко всей площади дома,1.258,0.866
10,отношение площади крыши ко всей площади дома,1.258,-0.866
6,площадь сплошного остекления,1.934,0.270
3,зона крыши,2.278,-0.862
1,площадь поверхности,2.660,-0.658
9,отношение остекленния ко всей площади дома,4.763,0.416
0,относительная компактность,7.594,0.622


In [21]:
bestfeatures = SelectKBest(f_regression, k=5)#todo поменять f_regression
fit = bestfeatures.fit(df_X, df_Y)
df_scores = pd.DataFrame(fit.scores_)
df_columns = pd.DataFrame(df_X.columns)
feature_scores = pd.concat([df_columns, df_scores, pd.DataFrame( corr["отопительная нагрузка"
].drop("отопительная нагрузка").values)], axis=1)
feature_scores.columns = ['Признак', 'Оценка', 'Коэффициент корреляции']
feature_scores=feature_scores.sort_values(by='Оценка')
display(feature_scores)

,Признак,Оценка,Коэффициент корреляции
5,ориентация,0.005,-0.003
7,распределение площади остекления,5.892,0.087
6,площадь сплошного остекления,60.156,0.270
9,отношение остекленния ко всей площади дома,160.372,0.416
2,площадь сплошной стены,200.728,0.456
0,относительная компактность,484.046,0.622
1,площадь поверхности,585.260,-0.658
3,зона крыши,2211.620,-0.862
10,отношение площади крыши ко всей площади дома,2295.956,-0.866
8,отношение стенной площади ко всей площади дома,2295.956,0.866


Сноска: реверс анализ типов данных

Исходя из здравого смысла и шестого чувства предположим, что
1. ориентация - это стороны света => есть потенцциальная возможность разбить на пару север-юг запад-восток (если влияние ветра не велико)
2. распределение площади остекления => куда выходят окна 0-не выходят никуда
3. площадь сплошного остекления. Размерность отличается от общей площади. 

In [22]:
display(feature_scores[:3])
display(df.head(3))
if feature_scores[:3]["Признак"].to_list() == [col for col in feature_scores[:3]["Признак"].to_list() if col in df.columns]:
    df.drop(feature_scores[:3]["Признак"].to_list(), axis=1, inplace=True)
display(df.head(3))
df_X = df.drop(columns='отопительная нагрузка')
df_Y = df['отопительная нагрузка']

,Признак,Оценка,Коэффициент корреляции
5,ориентация,0.005,-0.003
7,распределение площади остекления,5.892,0.087
6,площадь сплошного остекления,60.156,0.270


,относительная компактность,площадь поверхности,площадь сплошной стены,зона крыши,общая высота,ориентация,площадь сплошного остекления,распределение площади остекления,отопительная нагрузка,отношение стенной площади ко всей площади дома,отношение остекленния ко всей площади дома,отношение площади крыши ко всей площади дома
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,0.571,0.0,0.214
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,0.571,0.0,0.214
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,0.571,0.0,0.214


,относительная компактность,площадь поверхности,площадь сплошной стены,зона крыши,общая высота,отопительная нагрузка,отношение стенной площади ко всей площади дома,отношение остекленния ко всей площади дома,отношение площади крыши ко всей площади дома
0,0.98,514.5,294.0,110.25,7.0,15.55,0.571,0.0,0.214
1,0.98,514.5,294.0,110.25,7.0,15.55,0.571,0.0,0.214
2,0.98,514.5,294.0,110.25,7.0,15.55,0.571,0.0,0.214


<h5 style="text-align: center; font-size: 20px;">Преобразование данных</h5>

<h5 style="text-align: center; font-size: 20px;">Обработка текстовых и категориальных атрибутов</h5>

In [23]:
list_od_ordinary_columns = ["общая высота", "ориентация", "распределение площади остекления"]
for el in list_od_ordinary_columns:
    if el in df.columns:
        enc = OrdinalEncoder(dtype=np.int64)
        df[el] = enc.fit_transform(pd.DataFrame(df[el]))
        display(enc.categories_)


[array([3.5, 7. ])]

<h5 style="text-align: center; font-size: 20px;">Масштабирование</h5>

In [ ]:
#Код для масштабирования

<h5 style="text-align: center; font-size: 20px;">Стандартизация</h5>

In [ ]:
#код для  Стандартизации

<h5 style="text-align: center; font-size: 20px;">Нормализация</h5>

In [24]:
def scale(data_frame):
    df_normalized = pd.DataFrame(Normalizer().fit_transform(data_frame))
    #_ = list(df.columns)
    #_.remove('отопительная нагрузка')
    df_normalized.columns = data_frame.columns
    display(df_normalized)
    return df_normalized
df_normalized = scale(df)
df_base_normilized = scale(df_base)

,относительная компактность,площадь поверхности,площадь сплошной стены,зона крыши,общая высота,отопительная нагрузка,отношение стенной площади ко всей площади дома,отношение остекленния ко всей площади дома,отношение площади крыши ко всей площади дома
0,1.625e-03,0.853,0.488,0.183,0.002,0.026,9.477e-04,0.000e+00,3.554e-04
1,1.625e-03,0.853,0.488,0.183,0.002,0.026,9.477e-04,0.000e+00,3.554e-04
2,1.625e-03,0.853,0.488,0.183,0.002,0.026,9.477e-04,0.000e+00,3.554e-04
3,1.625e-03,0.853,0.488,0.183,0.002,0.026,9.477e-04,0.000e+00,3.554e-04
4,1.365e-03,0.855,0.483,0.186,0.002,0.032,8.576e-04,0.000e+00,3.298e-04
...,...,...,...,...,...,...,...,...,...
763,7.241e-04,0.887,0.388,0.249,0.000,0.020,4.950e-04,5.772e-07,3.182e-04
764,6.774e-04,0.883,0.402,0.241,0.000,0.018,4.967e-04,5.406e-07,2.980e-04
765,6.774e-04,0.883,0.402,0.241,0.000,0.018,4.967e-04,5.406e-07,2.980e-04
766,6.774e-04,0.883,0.402,0.241,0.000,0.018,4.967e-04,5.406e-07,2.980e-04


,относительная компактность,площадь поверхности,площадь сплошной стены,зона крыши,общая высота,ориентация,площадь сплошного остекления,распределение площади остекления,отопительная нагрузка
0,1.625e-03,0.853,0.488,0.183,0.012,0.003,0.000e+00,0.000,0.026
1,1.625e-03,0.853,0.488,0.183,0.012,0.005,0.000e+00,0.000,0.026
2,1.625e-03,0.853,0.488,0.183,0.012,0.007,0.000e+00,0.000,0.026
3,1.625e-03,0.853,0.488,0.183,0.012,0.008,0.000e+00,0.000,0.026
4,1.365e-03,0.855,0.483,0.186,0.011,0.003,0.000e+00,0.000,0.032
...,...,...,...,...,...,...,...,...,...
763,7.241e-04,0.887,0.388,0.249,0.004,0.006,4.525e-04,0.006,0.020
764,6.774e-04,0.883,0.402,0.241,0.004,0.002,4.370e-04,0.005,0.018
765,6.774e-04,0.883,0.402,0.241,0.004,0.003,4.370e-04,0.005,0.018
766,6.774e-04,0.883,0.402,0.241,0.004,0.004,4.370e-04,0.005,0.018


In [25]:
df_X = df_normalized.drop(columns='отопительная нагрузка')
df_Y = df_normalized['отопительная нагрузка']



In [26]:
TEST_SIZE = 0.2
VALID_SIZE = 0.1

SEED = 1337

assert VALID_SIZE*1.5 <= TEST_SIZE
VALID_SIZE = VALID_SIZE / TEST_SIZE

In [35]:
def var2str(var, vars_data=locals()):
    return [var_name for var_name in vars_data if id(var) == id(vars_data[var_name])]
    
def test(data_frame, data_type=0, degrees=[1, 2]):
    df_X = data_frame.drop(columns='отопительная нагрузка')
    df_Y = data_frame['отопительная нагрузка']

    X_train, X_test, Y_train, Y_test = train_test_split(df_X, df_Y, test_size=TEST_SIZE, random_state=SEED)
    X_test, X_valid, Y_test, Y_valid = train_test_split(X_test, Y_test, test_size=VALID_SIZE, random_state=SEED)
    print(f"frame = {var2str(data_frame)}")
    YY = [Y_train, Y_test, Y_valid]
    #display(X_train, X_test)
    for d in degrees:
        if d > 1:
            model_poly = PolynomialFeatures(degree=2)
            X_train = pd.DataFrame(model_poly.fit_transform(X_train))
            X_test = pd.DataFrame(model_poly.fit_transform(X_test))
            X_valid = pd.DataFrame(model_poly.fit_transform(X_valid))

        #print((X_train.shape))
        XX = [X_train, X_test, X_valid]

        rates = []
        model_lr = LinearRegression()
        moder_ridge = Ridge()
        #models = [model_lr, moder_ridge]
        models = [moder_ridge]
        names = ["LinearRegression", "Ridge"]
        res = []
        for m in models:
            m.fit(X_train, Y_train)
            res.append(m.predict(XX[data_type]))
        print(f"d = {d}")
        for i in range(len(models)):
            rates.append((np.sqrt(mean_squared_error(res[i], YY[data_type])), r2_score(YY[data_type], res[i])))
        
            print(f"{names[i]+' '*(20-len(names[i]))} \tRMSE = \t{rates[i][0]} \tR2_score = \t{rates[i][1]}")

datasets = [df_normalized]#, df, df_base_normilized, df_base]
for el in datasets:
    test(el)
for el in datasets:
    test(el, 1, [1,2,3,5])    

frame = ['el', 'df_normalized']
d = 1
LinearRegression     	RMSE = 	0.008186286175004467 	R2_score = 	0.7154382529571448
d = 2
LinearRegression     	RMSE = 	0.007783468540569838 	R2_score = 	0.7427537680060456
frame = ['el', 'df_normalized']
d = 1
LinearRegression     	RMSE = 	0.009068128689975502 	R2_score = 	0.6923195806761742
d = 2
LinearRegression     	RMSE = 	0.008588782423984954 	R2_score = 	0.7239881605549087
d = 3
LinearRegression     	RMSE = 	0.008364728613454933 	R2_score = 	0.7382008575421144


KeyboardInterrupt: 

In [ ]:
def test(data_frame, data_type=0, degrees=[1, 2, 3, 5]):
    df_X = data_frame.drop(columns='отопительная нагрузка')
    df_Y = data_frame['отопительная нагрузка']

    X_train, X_test, Y_train, Y_test = train_test_split(df_X, df_Y, test_size=TEST_SIZE, random_state=SEED)
    X_test, X_valid, Y_test, Y_valid = train_test_split(X_test, Y_test, test_size=VALID_SIZE, random_state=SEED)
    
    print(f"frame = {var2str(data_frame)}")
    YY = [Y_train, Y_test, Y_valid]

    best_rmse = float('inf') 
    best_r2 = -float('inf')  
    best_degree = None  
    best_model = None 
    best_data_type = data_type  

    for d in degrees:
        X_train_poly, X_test_poly, X_valid_poly = X_train, X_test, X_valid
        
        if d > 1:
            model_poly = PolynomialFeatures(degree=d)
            X_train_poly = pd.DataFrame(model_poly.fit_transform(X_train))
            X_test_poly = pd.DataFrame(model_poly.fit_transform(X_test))
            X_valid_poly = pd.DataFrame(model_poly.fit_transform(X_valid))

        XX = [X_train_poly, X_test_poly, X_valid_poly]
        
        rates = []
        model_ridge = Ridge(alpha=1.0)  # Гребневая регрессия
        models = [model_ridge]
        names = ["Ridge"]
        res = []
        
        for m in models:
            m.fit(X_train_poly, Y_train)
            res.append(m.predict(XX[data_type]))
        
        print(f"d = {d}")
        for i in range(len(models)):
            rmse = np.sqrt(mean_squared_error(res[i], YY[data_type]))
            r2 = r2_score(YY[data_type], res[i])
            rates.append((rmse, r2))
            print(f"{names[i]+' '*(20-len(names[i]))} \tRMSE = \t{rmse} \tR2_score = \t{r2}")
            
            
            if rmse < best_rmse or (rmse == best_rmse and r2 > best_r2):
                best_rmse = rmse
                best_r2 = r2
                best_degree = d
                best_model = m
                best_data_type = data_type

    return best_rmse, best_r2, best_degree, best_model, best_data_type


def select_best_model(datasets, degrees=[1, 2, 3, 5]):
    overall_best_rmse = float('inf')
    overall_best_r2 = -float('inf')
    overall_best_dataset = None
    overall_best_degree = None
    overall_best_model = None
    overall_best_data_type = None

    
    for data_frame in datasets:
        print(f"\nEvaluating dataset: {var2str(data_frame)}")
        for data_type in [0, 1]:  
            print(f"\nEvaluating for data_type {data_type}...")
            best_rmse, best_r2, best_degree, best_model, best_data_type = test(data_frame, data_type=data_type, degrees=degrees)
            

            if best_rmse < overall_best_rmse or (best_rmse == overall_best_rmse and best_r2 > overall_best_r2):
                overall_best_rmse = best_rmse
                overall_best_r2 = best_r2
                overall_best_dataset = data_frame
                overall_best_degree = best_degree
                overall_best_model = best_model
                overall_best_data_type = best_data_type

    print(f"\nBest overall model found:")
    print(f"Best RMSE = {overall_best_rmse}, Best R² = {overall_best_r2}")
    print(f"Best dataset: {var2str(overall_best_dataset)}")
    print(f"Best degree: {overall_best_degree}")




datasets = [df_normalized, df, df_base_normilized, df_base] 
select_best_model(datasets)



Evaluating dataset: ['df_normalized']

Evaluating for data_type 0...
frame = ['df_normalized']
d = 1
Ridge                	RMSE = 	0.008186286175004467 	R2_score = 	0.7154382529571448
d = 2
Ridge                	RMSE = 	0.007783468540569838 	R2_score = 	0.7427537680060456
d = 3
Ridge                	RMSE = 	0.007701926343953813 	R2_score = 	0.7481155279308532
d = 5
Ridge                	RMSE = 	0.007634721166859192 	R2_score = 	0.7524921176848838

Evaluating for data_type 1...
frame = ['df_normalized']
d = 1
Ridge                	RMSE = 	0.009068128689975502 	R2_score = 	0.6923195806761742
d = 2
Ridge                	RMSE = 	0.008588782423984954 	R2_score = 	0.7239881605549087
d = 3
Ridge                	RMSE = 	0.00847800523788234 	R2_score = 	0.7310621860987985
d = 5
Ridge                	RMSE = 	0.008385357315318571 	R2_score = 	0.7369079917404534

Evaluating dataset: ['df']

Evaluating for data_type 0...
frame = ['df']
d = 1
Ridge                	RMSE = 	3.859987268931033 	R2_scor

In [ ]:
df_base.head()

ipywidgets          8.1.5

matplotlib          3.10.0

numpy               2.1.3

pandas              2.2.3

plotly              5.24.1

seaborn             0.13.2

session_info        1.0.0

sklearn             1.6.1
